IMT 2200 - Introducción a Ciencia de Datos<br>
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

---

## Proyecto de Investigación – Terremotos en Chile





## 1. Objetivos

**-Obtener información de terremotos en Chile**<br>
**-Calcular los mayores terremotos por región**<br> 
**-Calcular las zonas de mayores riesgos para terremotos comparando la escala de mercalli con la de richer**<br> 
**-Crear un modelo que pueda predecir en base a las estadísticas los posibles lugares de riegos**<br>
**-Crear un promedio de tiempo en el que pueda predecir cada cuanto puede acontecer un terremoto**<br>


## 2. Datos

Los datos serán extraídos de https://www.usgs.gov/programs/earthquake-hazards/earthquakes. Principalmente del link, https://earthquake.usgs.gov/earthquakes/search/ . Donde se pueden filtrar los terremotos por zonas, año  y magnitud, generando un archivo .csv con la información.

In [23]:
import pandas as pd
from pathlib import Path

## 3. Desarrollo

La apertura del documento descargado podemos observar que el Data Frame generado cuenta con más de 10.000 filas, y con 22 columnas, con información de tiempo, ubicación, magnitud, entre otros.

In [24]:
ruta = Path("query.csv")
df = pd.read_csv(ruta) 
df.columns


Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10407 entries, 0 to 10406
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             10407 non-null  object 
 1   latitude         10407 non-null  float64
 2   longitude        10407 non-null  float64
 3   depth            10407 non-null  float64
 4   mag              10407 non-null  float64
 5   magType          10407 non-null  object 
 6   nst              4033 non-null   float64
 7   gap              5882 non-null   float64
 8   dmin             2799 non-null   float64
 9   rms              8082 non-null   float64
 10  net              10407 non-null  object 
 11  id               10407 non-null  object 
 12  updated          10407 non-null  object 
 13  place            10407 non-null  object 
 14  type             10407 non-null  object 
 15  horizontalError  2563 non-null   float64
 16  depthError       5446 non-null   float64
 17  magError    

In [26]:
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-09-01T13:48:01.759Z,-30.6105,-71.2446,61.209,4.6,mb,50.0,71.0,0.342,0.81,...,2025-09-01T17:26:31.440Z,"4 km WSW of Ovalle, Chile",earthquake,5.13,5.452,0.109,25.0,reviewed,us,us
1,2025-08-29T21:48:43.387Z,-18.9492,-69.4232,114.517,4.6,mb,53.0,103.0,0.616,0.98,...,2025-08-30T03:24:45.040Z,"40 km N of Camiña, Chile",earthquake,5.16,5.411,0.079,49.0,reviewed,us,us
2,2025-08-25T18:01:03.119Z,-21.7142,-68.4897,118.465,4.7,mb,44.0,63.0,0.704,1.19,...,2025-08-25T18:16:37.040Z,"59 km SSW of Ollagüe, Chile",earthquake,4.54,5.964,0.060,85.0,reviewed,us,us
3,2025-08-24T03:44:08.460Z,-19.3888,-69.2674,100.534,5.0,mb,59.0,104.0,0.287,1.04,...,2025-08-24T15:32:52.214Z,"18 km ESE of Camiña, Chile",earthquake,6.76,5.716,0.029,391.0,reviewed,us,us
4,2025-08-23T09:07:59.103Z,-32.5045,-71.5221,53.111,4.7,mb,49.0,133.0,0.527,0.84,...,2025-08-24T15:09:12.040Z,"27 km WSW of La Ligua, Chile",earthquake,2.93,5.576,0.097,32.0,reviewed,us,us
